<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Проверим-R2-на-искомых-данных" data-toc-modified-id="Проверим-R2-на-искомых-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Проверим R2 на искомых данных</a></span></li><li><span><a href="#Проверим-вектор-весов-после-изменения" data-toc-modified-id="Проверим-вектор-весов-после-изменения-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Проверим вектор весов после изменения</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.shape

(5000, 5)

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Проверим R2 на искомых данных

In [5]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [6]:
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                            test_size=0.4, random_state=12345)

In [7]:
model = LinearRegression().fit(features_train, target_train)
predictions = model.predict(features_test)

In [8]:
r2_score(target_test, predictions)

0.42375177725680335

### Проверим вектор весов после изменения

In [9]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

**Ответ:** веса совпадают

**Обоснование:** 

- w' = (Y^TY)^{-1} Y^Ty =
- = ((XP)^T(XP))^{-1} (XP)^Ty =
- = P^{-1}((XP)^{T}X)^{-1} (XP)^Ty =
- = P^{-1}((XP)^{T}X)^{-1} P^TX^Ty =
- = P^{-1}(P^TX^TX)^{-1} P^TX^Ty =
= P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty
Сократим

(P^{T})^{-1} P^T = P^{-1}(X^TX)^{-1} X^Ty

Произведем замену

w = (X^TX)^{-1}X^Ty = P^{-1}w

Тогда: w(y)= P^{-1}w

Подставим для X' и w' в формулу для расчета a':

a'=X'w' = XPP^{-1}w = Xw = a

Выражения тождественны. Векторы предсказаний совпали.

## Алгоритм преобразования

**Алгоритм**
* Отделяем целевой признак от набора данных
* Генерируем и фиксируем случайную квадратную матрицу в качестве ключа шифрования
* Производим матричное умножение набора данных и ключа шифровнаия
* Производим необходимые операции с датасетом
* Производим дешифрование с помощью матричного умножения преобразованного набора на обратную матрицу ключа шифрования

**Обоснование**
Произведение матрицы состоит из всех возможных комбинаций скалярных произведений вектор-строк матрицы Features и вектор-столбцов матрицы Random. Таким образом мы произведем умножение каждого набора признаков (строка-вектор) из Features на одинаковые наборы векторов (столбец - вектор) из Random и запишем их в новый вектор - строку. Соответственно итоговые веса каждого такого вектора будут примерно соотносится с весами набора признаков для каждого нового набора векторов в результирующей матрице.

## Проверка алгоритма

In [10]:
random_2 = []
for i in range (50):
    try:
          i = np.random.randint(0,5, size=(4,4))
          k = np.linalg.inv(i)
          one = np.eye(4)
          m = i @ k
          if m[3].tolist() == one[3].tolist():
              if m[2].tolist() == one[2].tolist():
                  if m[1].tolist() == one[1].tolist():
                      if m[0].tolist() == one[0].tolist():
                          random_2.append(i)
    except: print('Найдена сингулярная матрица')

In [11]:
print(random_2)

[array([[4, 0, 1, 4],
       [2, 4, 1, 4],
       [1, 0, 1, 0],
       [4, 4, 2, 0]]), array([[1, 2, 1, 0],
       [2, 4, 1, 2],
       [0, 4, 4, 4],
       [2, 0, 4, 0]]), array([[4, 0, 2, 3],
       [2, 4, 4, 1],
       [1, 3, 4, 1],
       [4, 0, 2, 4]])]


In [12]:
random = random_2[0]
print(random)

[[4 0 1 4]
 [2 4 1 4]
 [1 0 1 0]
 [4 4 2 0]]


In [13]:
inverse = np.linalg.inv(random)

In [14]:
random @ inverse

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [15]:
check = features.values @ random

In [16]:

check_train, check_test, target_c_train, target_c_test = train_test_split(check, target, 
                                                                            test_size=0.4, random_state=12345)

In [17]:
model = LinearRegression().fit(check_train, target_c_train)
predictions_c = model.predict(check_test)

In [18]:
r2_score(target_c_test, predictions_c)

0.42375177725680546

Метрика R2 совпадает с метрикой модели, обученной на искомых данных. Задача решена